<a href="https://colab.research.google.com/github/devasworski/Deeper-Networks-for-Image-Classification/blob/main/GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GoogleNet for Image Classification

Using GoogleNet on the MNIST and CIFAR datasets for image classification.

For the model definition, the code from [Khuyen Le](https://medium.com/mlearning-ai/implementation-of-googlenet-on-keras-d9873aeed83c) was used as a basis and then adjusted using the Guide from [Jai Faizy ](https://www.analyticsvidhya.com/blog/2018/10/understanding-inception-network-from-scratch/) and the architecture of the Model was then updated to [V2](https://towardsdatascience.com/a-simple-guide-to-the-versions-of-the-inception-network-7fc52b863202) 

## Init

In [8]:
from py import googLeNet
from py import datasets
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import ConfusionMatrixDisplay
from enum import Enum
import numpy as np
import sklearn.metrics
import matplotlib.pyplot as plt
from tensorflow import keras

### Helper functions

In [2]:
def plot_confusion_matrix(confusion_matrix):
  confusionMatrixDisplay = ConfusionMatrixDisplay(confusion_matrix, display_labels=range(10))
  fig, ax = plt.subplots(figsize=(40,40))
  confusionMatrixDisplay.plot(ax=ax,cmap = plt.get_cmap('Blues'), xticks_rotation='vertical')

### Enums

In [3]:
class Dataset(Enum):
    MNIST = 1
    CIFAR = 2
class Optimizer(Enum):
    Adam = 1
    SGD = 2
class Runtime(Enum):
    local = 1
    colab = 2

## Hyperparameters

In [4]:
EPOCHS = 20
LEARNING_RATE = 0.01
DATASET = Dataset.MNIST
RUNTIME = Runtime.colab
CHECKPOINT_PATH = "/content/drive/Othercomputers/My MacBook Pro/Semester B/Deep Learning and Computer Vision/Cousework/CW3/Checkpoints/GoogLeNet/"+('MNIST' if DATASET == Dataset.MNIST else 'CIFAR')+"/cp-{epoch:04d}.ckpt" if RUNTIME == Runtime.colab else "/Checkpoints/GoogLeNet/"+('MNIST' if DATASET == Dataset.MNIST else 'CIFAR')+"/cp-{epoch:04d}.ckpt"
OPTIMIZER = Optimizer.SGD

## Load Model

In [ ]:
(x_train, y_train), (x_val, y_val), (x_test, y_test) = datasets.getMnist() if DATASET == Dataset.MNIST else datasets.getCifar()

In [9]:
optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
if OPTIMIZER == Optimizer.SGD: 
    optimizer = keras.optimizers.SGD(learning_rate=LEARNING_RATE)

In [ ]:
model = googLeNet.googlenet(10)
model.summary()

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

## Train

### Define Checkpoint

In [ ]:
cp_callback = ModelCheckpoint(filepath=CHECKPOINT_PATH,verbose=1,save_weights_only=True,monitor='val_accuracy')

### Train Model

In [ ]:
model.fit(x=x_train, y=y_train, epochs=EPOCHS, verbose=1, validation_data=(x_test, y_test), batch_size=128, callbacks=cp_callback)

### Load Checkpoint

In [ ]:
epoch_num = EPOCHS
path = CHECKPOINT_PATH.format(epoch_num)
model.load_weights(path)

## Evaluate Model

In [ ]:
score = model.evaluate(x_test, y_test, batch_size=100)
print("Overall Accuracy:", score[1]*100,'%')

In [ ]:
pred = model.predict(x_test, batch_size=100)
confusion_matrix = sklearn.metrics.confusion_matrix(np.argmax(y_test, axis=1),np.argmax(pred, axis=1))
plot_confusion_matrix(confusion_matrix)